In [1]:
import pandas as pd 

ddd = pd.read_csv('csvs/linkedin_f.csv')

In [2]:
from webscraping import linkedin_driver

driver = linkedin_driver()

In [32]:
from webscraping import linkedin_page_go_to,linkedin_get_text_values_by_class,linkedin_get_href_by_class
from webscraping import linkedin_get_href_by_class,linkedin_driver, linkedin_get_elements_by_css
from time import sleep
from webscraping import linkedin_job_find_next_page_button, linkedin_click
from webscraping import By, find_technologies_in_string
from random import randint

login = r'https://www.linkedin.com/login?emailAddress=&fromSignIn=&fromSignIn=true&session_redirect=https%3A%2F%2Fuk.linkedin.com%2Fjobs%2Fview%2Fdata-engineer-at-iq-eq-3741507411%3FrefId%3Did3VRcGHCt8KUHuyFlp%252BRQ%253D%253D%26trackingId%3D0HBMkGcz00qUBabeUAttyA%253D%253D%26position%3D1%26pageNum%3D0%26trk%3Dpublic_jobs_jserp-result_search-card&trk=public_jobs_nav-header-signin'
linkedin_page_go_to(driver, login)

import pickle

# To load the password from the pickle file, you can use the following code:
with open('linkedin_user.pkl', 'rb') as file:
    username = pickle.load(file)
with open('linkedin_password.pkl', 'rb') as file:
    password = pickle.load(file)

user_field = driver.find_elements(By.ID, 'username')[0]
pass_field = driver.find_elements(By.ID, 'password')[0]

user_field.send_keys(username)
pass_field.send_keys(password)

element = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')
linkedin_click(element)

In [42]:
from webscraping import linkedin_page_go_to,linkedin_get_text_values_by_class,linkedin_get_href_by_class
from webscraping import linkedin_get_href_by_class,linkedin_driver, linkedin_get_elements_by_css
from time import sleep
from webscraping import linkedin_job_find_next_page_button, linkedin_click
from webscraping import By, find_technologies_in_string
from random import randint

data_list = []
counter = 0
for row in ddd.iterrows():
    counter+=1
    if counter>1:
        break
    ID = row[1][0]
    SOURCE = row[1][-1]
    # print(ID)
    linkedin_page_go_to(driver, row[1][5])
    sleep(randint(2,4))
    # sleep(1)
    try:
        lower,higher = None , None
        button = driver.find_element(By.CLASS_NAME, 'jobs-description__footer-button')
        linkedin_click(button)
        # print('clicking btton')
        items = driver.find_elements(By.CLASS_NAME,'jobs-box__html-content')[0].text
        print('found elements')
        # link = linkedin_get_href_by_class('topcard__org-name-link',driver)
        # print('finding link')
        # data = {'ID':ID, 'source':SOURCE, 'min_yearly_salary':lower,
        #         'max_yearly_salary':higher,'company_link':link,'description':items}
        # techs = find_technologies_in_string(items)
        # data.update(techs)
        # print('appending data')
        # data_list.append(data)
    except:
        pass
df_new = pd.DataFrame(data_list)
# df_new

found elements


In [43]:
items

"About the job\nCompany Description\n\nIQEQ is a preeminent service provider to the alternative asset industry. IQEQ works with managers in multiple capacities ranging from hedge fund, private equity fund, and mutual fund launches; private equity fund administration; advisory firm set-up, regulatory registration and infrastructure design; ongoing regulatory compliance (SEC, CFTC, and 40 Act); financial controls and operational support services; compliance and operational related projects and reviews; and outsourced CFO/controller and administration services to private equity fund investments – portfolio companies, real estate assets and energy assets. Our client base is growing, and our existing clients are engaging the firm across the spectrum of our services offerings.\n\nJob Description\n\nN.B. this role is based in Belfast.\n\nWe have a fantastic opportunity for an experienced Data Engineer to join our global team. This role will play a major part in the delivery of our Group Data 

In [4]:
lower,higher = None , None
close = driver.find_elements(By.ID, 'close-small')
if close:
    for closing in close:
        linkedin_click(closing)
        print('found close button')
sleep(randint(2,4))
button = driver.find_element(By.CLASS_NAME, 'jobs-description__footer-button')
linkedin_click(button)
print('button clicked')
items = driver.find_elements(By.CLASS_NAME,'show-more-less-html__markup')[0].text
print('finding elements')
link = linkedin_get_href_by_class('topcard__org-name-link',driver)
print('finding link')
data = {'ID':ID, 'source':SOURCE, 'min_yearly_salary':lower,
        'max_yearly_salary':higher,'company_link':link,'description':items}
techs = find_technologies_in_string(items)
data.update(techs)
print('appending data')
data_list.append(data)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=118.0.5993.71)
Stacktrace:
	GetHandleVerifier [0x01074DE3+43907]
	(No symbol) [0x01000741]
	(No symbol) [0x00EF32B0]
	(No symbol) [0x00F27958]
	(No symbol) [0x00F1FF9A]
	(No symbol) [0x00F42B5C]
	(No symbol) [0x00F1F9D6]
	(No symbol) [0x00F42DD4]
	(No symbol) [0x00F555CA]
	(No symbol) [0x00F42956]
	(No symbol) [0x00F1E17E]
	(No symbol) [0x00F1F32D]
	GetHandleVerifier [0x01325AF9+2865305]
	GetHandleVerifier [0x0136E78B+3163435]
	GetHandleVerifier [0x01368441+3138017]
	GetHandleVerifier [0x010FE0F0+605840]
	(No symbol) [0x0100A64C]
	(No symbol) [0x01006638]
	(No symbol) [0x0100675F]
	(No symbol) [0x00FF8DB7]
	BaseThreadInitThunk [0x75627BA9+25]
	RtlInitializeExceptionChain [0x7743BD3B+107]
	RtlClearBits [0x7743BCBF+191]


In [ ]:
close = driver.find_elements(By.ID, 'close-small')[0]
linkedin_click(close)
close

In [ ]:
link

In [ ]:
items = '''"MP DATA en résumé :\n\nMP DATA est une société spécialisée dans l’acquisition,
le traitement, et la valorisation des données. Depuis sa création en 2015, MP DATA accompagne
ses clients, majoritairement industriels, dans le management de leur performance et
l’exploitation de leurs données.\n\nLes collaborateurs, tous issus de grandes écoles,
incarnent au quotidien les valeurs d’Excellence, de Partage et d’Engagement. 
Ils associent savoir-faire technique, méthodologie et passion et mettent leurs compétences
au service de missions et projets au sein de grands groupes français.\n\nMP 
DATA accompagne ses clients sur toute la chaine au travers de 3 pôles d’expertise :
Conseil et Stratégie, Infrastructure & CloudOPS, Data Science.\n\nChez MP DATA, les 
équipes commerciales cherchent des missions en fonction des envies des collaborateurs
et non pas l’inverse. Les consultants sont accompagnés dans tous leurs projets, de 
la mobilité géographique, au changement de junior secteur d’activité en passant par le
développement de nouvelles compétences.\nRejoindre MP DATA, c’est la garantie
de travailler sur des sujets passionnants avec un cadre technique fort.\nMission 
:\n\nMP DATA, recrute un développeur, avec des affinités pour le métier de Data
Engineer ou un Data Engineer afin de travailler pour nous rejoindre. Accompagné
par un Lead Data Engineer, vous aurez l’occasion de monter en compétences sur
la manipulation de gros volumes de données.\n\nLocalisation : Cannes,
Antibes, Nice.\nTélétravail : À définir\n\nProfil recherché 
:\n\nIngénieur-e issu-e de grande école, spécialisé-e en développement
informatique / data engineering, vous justifiez de deux ou trois ans 
d’expérience professionnelle au cours desquelles vous avez pu développer
les compétences techniques suivantes :\n\nLangages : Scala,
Python\nFramework : Spark (connaissance de DataBricks),
Hadoop (connaissance de Map Reduce)\nTechnologies de stockage 
: Snowflake / Scality\nEnvironnement : Azure\n\nVous êtes reconnu(e) 
pour votre autonomie, votre excellent relationnel et votre capacité à être force de proposition
.\n\nProcess de recrutement :\n\nPremier échange en visio de 30 minutes.\nEntretien technique 
avec notre direction technique.\nÉchange avec l'équipe\nBienvenue dans nos équipes !"'''

In [ ]:
from webscraping import find_technologies_in_string
# Test the function
data = find_technologies_in_string(items)

data